In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [2]:
# Chrome 옵션 설정
options = Options()
options.binary_location =  ".\\chrome-win64\\chrome.exe"
options.add_argument("--no-sandbox") # sandbox 설정
options.add_argument("--disable-dev-shm-usage") # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # 화면크기설정 , 반응형 웹 대비
# options.add_argument("--disable-gpu") # GPU 비활성화

# ChromeDriver 설정
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.starbucks.co.kr/index.do")
print(driver.title)

# 요소찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
menu_option = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a")

# 마우스 호버 엑션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

# 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(3)
print("매장검색중")

# 입력창 선택 후 검색
driver.find_element(By.CSS_SELECTOR,"#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a").click() # '지역검색' 을 선택
print("지역검색중")
time.sleep(2)
driver.find_element(By.CSS_SELECTOR,"#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(10) > a").click() # '경남'을 클릭
print("경남검색중")
time.sleep(2)
driver.find_element(By.CSS_SELECTOR,"#mCSB_2_container > ul > li:nth-child(1) > a").click()
print("경남전체로드중")
time.sleep(2)
html = driver.page_source
# print(html) 

Starbucks Korea
매장검색중
지역검색중
경남검색중
경남전체로드중


In [4]:
# 현재 페이지 정보 가져오기
star_list=[]
soup = bs(html, 'html.parser')
# 본문내용 가져오기
contents = soup.select("#mCSB_3_container > ul > li")

# test = contents[0]
# print(test.select_one('p.result_details').text, '/', test.select_one('i.pin_generalDT').text)
# print(test['data-name'], '/', test['data-lat'], '/', test['data-long'])

for c in contents:
    result = ( c['data-name'], c.select_one('p.result_details').text , c.select_one('i')['class'][0], c['data-lat'],c['data-long'] )
    star_list.append(result)
    
print(star_list)

[('김해구산DT', '경상남도 김해시 김해대로 1918 (구산동)1522-3232', 'pin_generalDT', '35.255314991', '128.866384281'), ('김해진영DT', '경상남도 김해시 진영읍 여래로20번길 34-26 1522-3232', 'pin_generalDT', '35.3060040933565', '128.731753216114'), ('신세계김해', '경상남도 김해시 김해대로 2232 신세계백화점 1층1522-3232', 'pin_general', '35.22848288', '128.873045617'), ('김해이마트', '경상남도 김해시 김해대로 2232 김해이마트 1층1522-3232', 'pin_general', '35.229997434', '128.872135886'), ('김해대로DT', '경상남도 김해시 김해대로 2465 1522-3232', 'pin_generalDT', '35.227992', '128.896545'), ('김해외동DT', '경상남도 김해시 분성로 146 (외동)1522-3232', 'pin_generalDT', '35.22962555422071', '128.86294818906367'), ('김해율하', '경상남도 김해시 율하카페길 77 (관동동) 1층1522-3232', 'pin_general', '35.176597', '128.812782'), ('김해삼계', '경상남도 김해시 가야로 182 (삼계동)1522-3232', 'pin_general', '35.261527', '128.871614'), ('김해내동', '경상남도 김해시 경원로 67 (내동)1522-3232', 'pin_general', '35.2371519', '128.86752079999996'), ('서김해IC DT', '경상남도 김해시 금관대로 820-22 (명법동)1522-3232', 'pin_generalDT', '35.2038771973106', '128.837970580838'), ('김해주촌DT', '경상남도 

In [7]:
# oracle연결
import oracledb

try:
    conn = oracledb.connect(
        service_name = 'XEPDB1',
        host = "localhost",
        port = "1521",
        user="PKNU",
        password="1234"
    )
    print("Oracle이 성공적으로 연결되었습니다!")
    cursor = conn.cursor()
except Exception as e:
    print("연동실패! ",e)

Oracle이 성공적으로 연결되었습니다!


In [8]:
# 테이블 생성
query = """
    BEGIN
        EXECUTE IMMEDIATE '
            CREATE TABLE starbucks2 (
                id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                name VARCHAR2(50),
                address VARCHAR2(200),
                DT VARCHAR2(50),
                lat float,
                longi float
                )';
        EXCEPTION
            WHEN OTHERS THEN
                IF SQLCODE != -955 THEN
                    RAISE;
                END IF;
            END;
"""
cursor.execute(query)
print("테이블 생성완료")

테이블 생성완료


In [12]:
# Create 여러개의 데이터 삽입
star_list = []
que = "INSERT INTO starbucks2 (name, address, dt, lat, longi) VALUES (:1, :2, :3, :4, :5)"
contents = soup.select("#mCSB_3_container > ul > li")
for c in contents:
    result = (c['data-name'], c.select_one('p.result_details').text , c.select_one('i')['class'][0], c['data-lat'], c['data-long'])
    cursor.execute(que, result)
    star_list.append(result)
conn.commit()
print("여러개의 데이터 삽입 완료")

여러개의 데이터 삽입 완료


In [13]:
star_list

[('김해구산DT',
  '경상남도 김해시 김해대로 1918 (구산동)1522-3232',
  'pin_generalDT',
  '35.255314991',
  '128.866384281'),
 ('김해진영DT',
  '경상남도 김해시 진영읍 여래로20번길 34-26 1522-3232',
  'pin_generalDT',
  '35.3060040933565',
  '128.731753216114'),
 ('신세계김해',
  '경상남도 김해시 김해대로 2232 신세계백화점 1층1522-3232',
  'pin_general',
  '35.22848288',
  '128.873045617'),
 ('김해이마트',
  '경상남도 김해시 김해대로 2232 김해이마트 1층1522-3232',
  'pin_general',
  '35.229997434',
  '128.872135886'),
 ('김해대로DT',
  '경상남도 김해시 김해대로 2465 1522-3232',
  'pin_generalDT',
  '35.227992',
  '128.896545'),
 ('김해외동DT',
  '경상남도 김해시 분성로 146 (외동)1522-3232',
  'pin_generalDT',
  '35.22962555422071',
  '128.86294818906367'),
 ('김해율하',
  '경상남도 김해시 율하카페길 77 (관동동) 1층1522-3232',
  'pin_general',
  '35.176597',
  '128.812782'),
 ('김해삼계',
  '경상남도 김해시 가야로 182 (삼계동)1522-3232',
  'pin_general',
  '35.261527',
  '128.871614'),
 ('김해내동',
  '경상남도 김해시 경원로 67 (내동)1522-3232',
  'pin_general',
  '35.2371519',
  '128.86752079999996'),
 ('서김해IC DT',
  '경상남도 김해시 금관대로 820-22 (명법동)1522

In [14]:
columns = ['매장명','주소','DT','위도','경도']
sb = pd.DataFrame(star_list, columns=columns)
sb.head()

,매장명,주소,DT,위도,경도
0,김해구산DT,경상남도 김해시 김해대로 1918 (구산동)1522-3232,pin_generalDT,35.255314991,128.866384281
1,김해진영DT,경상남도 김해시 진영읍 여래로20번길 34-26 1522-3232,pin_generalDT,35.3060040933565,128.731753216114
2,신세계김해,경상남도 김해시 김해대로 2232 신세계백화점 1층1522-3232,pin_general,35.22848288,128.873045617
3,김해이마트,경상남도 김해시 김해대로 2232 김해이마트 1층1522-3232,pin_general,35.229997434,128.872135886
4,김해대로DT,경상남도 김해시 김해대로 2465 1522-3232,pin_generalDT,35.227992,128.896545


In [15]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
csv_path = f'ref/starbucks2_{timestamp}.csv'
sb.to_csv(csv_path, index=True)
print("csv 파일 생성완료")

csv 파일 생성완료


In [16]:
import pandas as pd
import folium
file_path = 'ref/starbucks2_20250414155513.csv'
data = pd.read_csv(file_path)
data = data[['매장명', '위도', '경도', '주소', 'DT']]
data.head()

,매장명,위도,경도,주소,DT
0,김해구산DT,35.255315,128.866384,경상남도 김해시 김해대로 1918 (구산동)1522-3232,pin_generalDT
1,김해진영DT,35.306004,128.731753,경상남도 김해시 진영읍 여래로20번길 34-26 1522-3232,pin_generalDT
2,신세계김해,35.228483,128.873046,경상남도 김해시 김해대로 2232 신세계백화점 1층1522-3232,pin_general
3,김해이마트,35.229997,128.872136,경상남도 김해시 김해대로 2232 김해이마트 1층1522-3232,pin_general
4,김해대로DT,35.227992,128.896545,경상남도 김해시 김해대로 2465 1522-3232,pin_generalDT


In [17]:
fixdata = data[['매장명', '위도', '경도']]
fixdata.head()

,매장명,위도,경도
0,김해구산DT,35.255315,128.866384
1,김해진영DT,35.306004,128.731753
2,신세계김해,35.228483,128.873046
3,김해이마트,35.229997,128.872136
4,김해대로DT,35.227992,128.896545


In [20]:
coordinates = fixdata[['위도', '경도']].to_numpy()
place = fixdata['매장명'].to_numpy()

# 중심좌표 설정(기본위치)
center_lat, center_lon = 35.262179, 129.091325

# 지도생성
m = folium.Map(location = [center_lat, center_lon], zoom_start=12)
store_type = data['DT']

# lat, lon = coordinates[0]
for i, (lat, lon) in enumerate(coordinates):
    if store_type[i] == "pin_generalDT":
        ccolor = "blue"
        fcolor = "skyblue"
    elif store_type[i] == "pin_general":
        ccolor = "green"
        fcolor = "yellow"
    else:
        ccolor = "red"
        fcolor = "orange"

    folium.Marker(
        location = [lat, lon],
        # popup='🐰',
        # tooltip = place[i]
        icon=folium.Icon(color=ccolor, icon='coffee', prefix='fa')
    ).add_to(m)    # 지도 생성 후 원을 그리고 지도에 붙임
m